Data Access

In [0]:
spark.conf.set("fs.azure.account.auth.type.de1storagedatalake.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.de1storagedatalake.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.de1storagedatalake.dfs.core.windows.net", "<your_key>")
spark.conf.set("fs.azure.account.oauth2.client.secret.de1storagedatalake.dfs.core.windows.net", "<your_secret>")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.de1storagedatalake.dfs.core.windows.net", "<your_endpoint>")

Data Loading

In [0]:
dbutils.fs.ls("abfss://bronze@de1storagedatalake.dfs.core.windows.net")

In [0]:
# df_ = {}
for f in dbutils.fs.ls("abfss://bronze@de1storagedatalake.dfs.core.windows.net"): 
    # print(f.name)
    name = f.name.replace('/','')
    # print(name)
    x = "df_"+ name
    print(x)
  

In [0]:
df_calendar = spark.read.format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load(f"abfss://bronze@de1storagedatalake.dfs.core.windows.net/calendar")

df_customers = spark.read.format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load(f"abfss://bronze@de1storagedatalake.dfs.core.windows.net/customers")
df_product_categories = spark.read.format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load(f"abfss://bronze@de1storagedatalake.dfs.core.windows.net/product_categories")
df_product_subcategories = spark.read.format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load(f"abfss://bronze@de1storagedatalake.dfs.core.windows.net/product_subcategories")
df_products = spark.read.format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load(f"abfss://bronze@de1storagedatalake.dfs.core.windows.net/products")

df_returns = spark.read.format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load(f"abfss://bronze@de1storagedatalake.dfs.core.windows.net/returns")

df_sales_2015 = spark.read.format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load(f"abfss://bronze@de1storagedatalake.dfs.core.windows.net/sales_2015")

df_sales_2016 = spark.read.format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load(f"abfss://bronze@de1storagedatalake.dfs.core.windows.net/sales_2016")

df_sales_2017 = spark.read.format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load(f"abfss://bronze@de1storagedatalake.dfs.core.windows.net/sales_2017")

df_territories = spark.read.format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load(f"abfss://bronze@de1storagedatalake.dfs.core.windows.net/territories")


In [0]:
# Select the common columns from each dataframe before performing the union
df_sales_2015_selected = df_sales_2015.select('OrderDate', 'StockDate', 'OrderNumber', 'ProductKey', 'CustomerKey', 'TerritoryKey', 'OrderLineItem', 'OrderQuantity')
df_sales_2016_selected = df_sales_2016.select('OrderDate', 'StockDate', 'OrderNumber', 'ProductKey', 'CustomerKey', 'TerritoryKey', 'OrderLineItem', 'OrderQuantity')
df_sales_2017_selected = df_sales_2017.select('OrderDate', 'StockDate', 'OrderNumber', 'ProductKey', 'CustomerKey', 'TerritoryKey', 'OrderLineItem', 'OrderQuantity')

# Perform the union on the dataframes with the same schema
df_sales = df_sales_2015_selected.union(df_sales_2016_selected).union(df_sales_2017_selected)

Transformations

In [0]:
df_calendar.display()
df_customers.display()
df_product_categories.display()
df_product_subcategories.display()
df_products.display()
df_returns.display()
df_sales_2015.display()
df_sales_2016.display()
df_sales_2017.display()
df_territories.display()

In [0]:
from pyspark.sql.functions import month, year
df_calendar = df_calendar.withColumn("Month", month(df_calendar.Date))\
    .withColumn("Year", year(df_calendar.Date))

In [0]:
df_calendar.display()

In [0]:
df_calendar.write.format("parquet")\
    .mode("append")\
    .option("path", f"abfss://silver@de1storagedatalake.dfs.core.windows.net/calendar")\
    .save()

In [0]:
from pyspark.sql.functions import concat_ws, lit

df_customers = df_customers.withColumn(
    "fullname", 
    concat_ws(' ',df_customers.Prefix, df_customers.FirstName, df_customers.LastName)
)
display(df_customers)

In [0]:
df_customers.write.format("parquet")\
    .mode("append")\
    .option("path", f"abfss://silver@de1storagedatalake.dfs.core.windows.net/customers")\
    .save()

In [0]:
df_product_categories.write.format("parquet")\
    .mode("append")\
    .option("path", f"abfss://silver@de1storagedatalake.dfs.core.windows.net/product_categories")\
    .save()

In [0]:
df_product_subcategories.write.format("parquet")\
    .mode("append")\
    .option("path", f"abfss://silver@de1storagedatalake.dfs.core.windows.net/product_subcategories")\
    .save()

In [0]:
df_products = spark.read.format("csv")\
        .option("header", True)\
        .option("inferSchema", True)\
        .load(f"abfss://bronze@de1storagedatalake.dfs.core.windows.net/products")
df_products.display()

In [0]:
from pyspark.sql.functions import split
df_products = df_products.withColumn("ProductSKU", split(df_products.ProductSKU, '-').getItem(0))\
                .withColumn("ProductName", split(df_products.ProductName, ',').getItem(0))

df_products.display()

In [0]:
df_products.write.format("parquet")\
    .mode("append")\
    .option("path", f"abfss://silver@de1storagedatalake.dfs.core.windows.net/products")\
    .save()

In [0]:
df_returns.write.format("parquet")\
    .mode("append")\
    .option("path", f"abfss://silver@de1storagedatalake.dfs.core.windows.net/returns")\
    .save()

In [0]:
df_territories.write.format("parquet")\
    .mode("append")\
    .option("path", f"abfss://silver@de1storagedatalake.dfs.core.windows.net/territories")\
    .save()

In [0]:
df_sales.display()

In [0]:
from pyspark.sql.functions import to_timestamp, regexp_replace, col

df_sales = df_sales.withColumn("StockDate", to_timestamp(df_sales.StockDate, "MM-dd-yyyy"))\
                             .withColumn("OrderNumber", regexp_replace(df_sales.OrderNumber, 'S', 'T'))\
                             .withColumn("Multiply", col("OrderLineItem") * col("OrderQuantity"))
display(df_sales)

In [0]:
from pyspark.sql.functions import count

df_sales.groupBy("OrderDate").agg(pyspark.sql.functions.count("OrderNumber").alias("Total_Order")).orderBy("Total_Order", ascending=False).display()


In [0]:
df_product_categories.display()

In [0]:
df_territories.display()

In [0]:
df_sales.write.format("parquet")\
    .mode("append")\
    .option("path", f"abfss://silver@de1storagedatalake.dfs.core.windows.net/sales")\
    .save()